# Fútbol bayesiano

Carga de librerías

In [1]:
import os
import yaml
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

Se definen rutas relativas del archivo

In [2]:
CURRENT = os.getcwd()
ROOT = os.path.dirname(CURRENT)

Cargar en memoria archivo de configuración

In [3]:
with open(os.path.join(ROOT, 'config.yml'), 'r') as file_:
    config = yaml.safe_load(file_)
file_.close()

Carga en memoria de los datos

In [15]:
df_ = (
    # Carga de datos crudos
    pd.read_csv(
        os.path.join(ROOT, config['datos']['raw'])
        ,encoding='latin1')
    # Filtrar columnas de interés
    .filter(items=[
        'date'
        ,'tournament'
        ,'home_team'
        ,'away_team'
        ,'home_team_continent'
        ,'away_team_continent'
        ,'home_team_score'
        ,'away_team_score'
    ])
    # Transformación a columnas existentes
    .assign(
        date = lambda df_: pd.to_datetime(df_.date)
        ,home_team = lambda df_: pd.Categorical(df_.home_team)
        ,away_team = lambda df_: pd.Categorical(df_.away_team)
        ,home_team_continent = lambda df_: pd.Categorical(df_.home_team_continent)
        ,away_team_continent = lambda df_: pd.Categorical(df_.away_team_continent)
    )
    # Asignación de columnas computadas
    .assign(
        result = lambda df_: (df_.home_team_score - df_.away_team_score).apply(np.sign)
    )
)

df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 23921 non-null  datetime64[ns]
 1   tournament           23921 non-null  object        
 2   home_team            23921 non-null  category      
 3   away_team            23921 non-null  category      
 4   home_team_continent  23921 non-null  category      
 5   away_team_continent  23921 non-null  category      
 6   home_team_score      23921 non-null  int64         
 7   away_team_score      23921 non-null  int64         
 8   result               23921 non-null  int64         
dtypes: category(4), datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


Modelo en STAN

Primero se definen algunas funciones útiles para la creación y manipulación de archivos de `stan`, y se crea un proyecto nuevo para la primera iteración del modelo jerárquico.

In [4]:
import BFG3000 as bfg

In [5]:
name = 'dinam_iter1'
sp_ = bfg.StanProject(name)
dir_, model_ = sp_.create_stan_project(dir='modelos')

Proyecto creado


Una vez creado y editado el proyecto de STAN, se declara el siguiente bloque de código para mantener estructura en la compilación del modelo y los archivos que éste genera.

In [ ]:
# Compilar el modelo
compile_ = CmdStanModel(
    model_name=name
    ,stan_file=model_
    ,compile=True)

Se delcara un diccionario de datos para proporcionar los datos a stan

In [ ]:
stan_data_ = {
    'n_matches': df_.shape[0]
    ,'goals_home': df_.home_team_score.values
    ,'goals_away': df_.away_team_score.values
}

Finalmente, se ejecuta el modelo y se calibra. Este paso es recursivo y se tiene que ejecutar tantas veces sean necesarias de modo que las cadenas generadas por el muestreador tengan diagnósticos buenos.

In [ ]:
fit_ = compile_.sample(
    data=stan_data_
    ,chains=4
    ,iter_warmup=1_250
    ,iter_sampling=5000
    ,show_progress=True
    ,save_warmup=False
)

21:46:33 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

21:54:30 - cmdstanpy - INFO - CmdStan done processing.
21:54:30 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: normal_lpdf: Scale parameter is -0.759836, but must be positive! (in '/Users/eduardo/Documents/Maestría/Bayesiana/futbol_bayes/modelos/iter2_jera/iter2_jera.stan', line 35, column 4 to column 47)
	Exception: normal_lpdf: Scale parameter is -0.392458, but must be positive! (in '/Users/eduardo/Documents/Maestría/Bayesiana/futbol_bayes/modelos/iter2_jera/iter2_jera.stan', line 35, column 4 to column 47)
	Exception: normal_lpdf: Scale parameter is -1.95639, but must be positive! (in '/Users/eduardo/Documents/Maestría/Bayesiana/futbol_bayes/modelos/iter2_jera/iter2_jera.stan', line 35, column 4 to column 47)
	Exception: normal_lpdf: Scale parameter is -0.137063, but must be positive! (in '/Users/eduardo/Documents/Maestría/Bayesiana/futbol_bayes/modelos/iter2_jera/iter2_jera.stan', line 35, column 4 to column 47)
	Exception: normal_lpdf: Scale parameter is -